# Code for generating and running PH of noisy networks - dev

In [3]:
# Load packages

using Pkg
using Statistics
using Plots
using LinearAlgebra
using Distances
using Eirene
using StatsBase
using Random
using Distributions
using JLD
using MAT

include("helpers.jl")
include("graph_models.jl")




make_dev_Geometric_configuration_model (generic function with 1 method)

## Choose weighed network and generate replicates

--------------------------------
## CHANGE PARAMETERS HERE
---------------------------------

In [4]:
## Pick from one of the graph models and generate out to THRESH edge density

const NREPS = 50
const RHO = 0.5      # Threshold at edge density = rho
const NNODES = 70
const SAVEFIGS = 1  # Boolean to save figure pdfs
const SAVEDATA = 1    # Boolean to save data  
const MAXDIM = 5

# for geometricConf
const P = 0.01
const SCALE_FACTOR = 100

# for RG
const DIMES = 3

# for discreteUniformConf
const A = 0
const B = 1000

# for cosineFeometric
const DIMS = 3


# Graph model name one of "geometricConf", "IID" , "RG", "discreteUniformConf", "cosineGeometric", "RL", "SWBM"
const GRAPH_MODEL_NAME = "RG"

"RG"

## Create weighted networks

In [5]:

# Construct weighted network with and without noise, with and without edge ordering
weighted_graph_array, weighted_graph_array_iidNoise, weighted_graph_array_iidNoise_ord, betti_file_name, parameters = construct_graphs(NNODES,NREPS,RHO,P,SCALE_FACTOR, DIMS, A, B, GRAPH_MODEL_NAME)
    


# Plot an example weighted graph and after we add noise

gr()

test_mat = 4
p0a = heatmap(weighted_graph_array[:,:,test_mat], aspect_ratio = :equal)
p0b = heatmap(weighted_graph_array_iidNoise[:,:,test_mat], aspect_ratio = :equal)
p0c = heatmap(weighted_graph_array_iidNoise_ord[:,:,test_mat], aspect_ratio = :equal)

plot(p0a,p0b,p0c,layout = (1,3))


if SAVEFIGS == 1
    savefig("../Figures/$(betti_file_name).pdf")
end

if SAVEDATA == 1
    save("../processed_data/$(betti_file_name)_graphs.jld",
     "weighted_graph_array", weighted_graph_array,
     "weighted_graph_array_iidNoise", weighted_graph_array_iidNoise,
     "parameters", parameters)
end



Naming files RG_500_05_700_30


## Running Eirene

In [21]:
# Loop through replicates and run eirene

nEdges = binomial(NNODES,2)
barcode_array = Array{Array{Float64}}(undef,NREPS,3)
betticurve_array = zeros(nEdges,MAXDIM,NREPS)
beginTime = time()



for rep = 1:NREPS
    
    G_i = weighted_graph_array_iidNoise_ord[:,:,rep]
    
    C = Eirene.eirene(G_i,model = "vr", maxdim = MAXDIM, record = "none")
    println("finished running eirene")
    for k in collect(1:MAXDIM)
        
        barcode_array[rep,k] = barcode(C,dim=k)
        betti_curve_i = betticurveFromBarcode(barcode_array[rep,k],nEdges)
        betticurve_array[:,k,rep] = betti_curve_i[1,:]
        
    end 
   
    if rep%10 == 0
        printstyled("\nRun $(rep) finished.\n",color = :cyan)
    end


end



endTime = time() - beginTime
printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

if SAVEDATA == 1
    save("../processed_data/$(betti_file_name)_eirene_output.jld",
        "barcode_array", barcode_array,
        "betticurve_array", betticurve_array)
end


printstyled("\nDone :)", color = :blue)


finished running eirene
finished running eirene
finished running eirene
finished running eirene
finished running eirene
Finished computing persistent homology, elapsed time 6.946272134780884 seconds
Done :)

In [23]:
## Plot average betti curves

betticurve_avg = dropdims(mean(betticurve_array,dims=3), dims = 3)
betticurve_std = dropdims(std(betticurve_array,dims=3), dims = 3)

x_axis = collect(1:nEdges)./nEdges

p1a = plot(x_axis,betticurve_avg[:,1],linewidth = 1.5, ribbon = betticurve_std[:,1], label = "Betti_1")
for d = 2:MAXDIM
    plot!(x_axis,betticurve_avg[:,d],linewidth = 1.5, ribbon = betticurve_std[:,d], label = "Betti_$(d)")
end

title!("$(betti_file_name)")
ylabel!("Betti_k")
xlabel!("Edge Density")

plot(p1a,layout = (1,1))

if SAVEFIGS == 1
    savefig("../Figures/$(betti_file_name).pdf")
end


In [7]:
using LightGraphs
using PyCall

nx = pyimport("networkx")

┌ Info: Precompiling LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d]
└ @ Base loading.jl:1242
┌ Info: Precompiling PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1242


PyObject <module 'networkx' from '/opt/conda/lib/python3.7/site-packages/networkx/__init__.py'>

In [8]:
gmn = "test"
a = 2
b = 3
c = [2,3,4]
d = "$(gmn)_$(a)_$(b)_$(c...)"
e = 5
f = "$(d)_$(e)"


"test_2_3_234_5"

In [9]:
macro get_var_name(arg)
   string(arg)
end

variablex  = 6

a = @Name(variablex)

LoadError: UndefVarError: @Name not defined

In [10]:
replace("4.32", "." => "")

"432"